diabetic 数据集 BT-TWD 可行性实验

本 notebook 按步骤运行：加载配置 → 读取数据 → 预处理 → 桶树划分 → 基线与 BTTWD k 折实验 → 桶级分析。

In [1]:
# 步骤0：环境与路径设置
import os, sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

# 将项目根目录加入路径，便于导入 bttwdlib
root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if root_path not in sys.path:
    sys.path.append(root_path)

from bttwdlib import (
    load_yaml_cfg,
    show_cfg,
    load_dataset,
    prepare_features_and_labels,
    BucketTree,
    run_kfold_experiments,
    log_info,
    set_global_seed,
)

cfg_path = Path(root_path) / "configs" / "diabetic_bttwd.yaml"
cfg = load_yaml_cfg(cfg_path)
set_global_seed(cfg.get('SEED', {}).get('global_seed', 42))
log_info('【步骤0摘要】环境准备完毕，路径与随机种子已设置。')

【INFO】【2025-12-12 21:45:02】【配置加载】已读取 e:\yan\组\三支决策\机器学习\BT_TWD\configs\diabetic_bttwd.yaml
【INFO】【2025-12-12 21:45:05】【步骤0摘要】环境准备完毕，路径与随机种子已设置。


In [2]:
# 步骤1：加载配置
show_cfg(cfg)
log_info('【步骤1摘要】配置文件加载完成，关键参数检查通过。')

【INFO】【2025-12-12 21:45:05】【配置-数据】数据集=diabetic, k折=5, 目标列=readmitted, 正类="<30"
【INFO】【2025-12-12 21:45:05】【配置-BTTWD】阈值模式=bucket_wise, 全局模型=xgb, 桶内模型=none, 后验估计器(兼容字段)=logreg
【INFO】【2025-12-12 21:45:05】【配置-基线】LogReg启用=True, RandomForest启用=True, KNN启用=True, XGBoost启用=True
【INFO】【2025-12-12 21:45:05】【步骤1摘要】配置文件加载完成，关键参数检查通过。


In [3]:
# 步骤2：加载原始数据
df_raw, target_col_model = load_dataset(cfg)  # 这里返回的是用于建模的标签列，例如 "label"

display(df_raw.head())
print("用于建模的标签列:", target_col_model)

# 1）画 0/1 标签（流失/未流失）的比例
class_counts = df_raw[target_col_model].value_counts(normalize=True)
ax = class_counts.plot(kind='bar', title='流失 vs 未流失比例')
plt.ylabel('比例')

fig_path = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'class_distribution.png')
os.makedirs(os.path.dirname(fig_path), exist_ok=True)
plt.savefig(fig_path, bbox_inches='tight')
plt.close()

# 2）如果想看原始标签列的分布，可以另外单独分析：
raw_target_col = cfg['DATA']['target_col']  # 这里是原始标签列
print("原始目标列:", raw_target_col)
print(df_raw[raw_target_col].describe())

log_info('【步骤2摘要】Telco Churn 原始数据加载与基本统计完成。')


【INFO】【2025-12-12 21:45:06】【数据加载】文本表格 ..\data\diabetic\diabetic_data.csv 已读取，样本数=101766，列数=50
【INFO】【2025-12-12 21:45:06】【数据加载】糖尿病数据集已读取，标签已二值化，样本数=101766，正类比例=11.16%
【INFO】【2025-12-12 21:45:06】【数据集信息】名称=diabetic，样本数=101766，目标列=readmitted，正类比例=11.16%


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,0
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,0
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,0
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,0
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,0


用于建模的标签列: readmitted
原始目标列: readmitted
count    101766.000000
mean          0.111599
std           0.314874
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: readmitted, dtype: float64
【INFO】【2025-12-12 21:45:06】【步骤2摘要】Telco Churn 原始数据加载与基本统计完成。


In [4]:
# 步骤3：预处理与特征工程
X, y, meta = prepare_features_and_labels(df_raw, cfg)
log_info(f'【预处理】编码特征维度={X.shape[1]}，样本数={X.shape[0]}')
log_info(f"【步骤3摘要】特征预处理完成：连续={len(meta['continuous_cols'])}，类别={len(meta['categorical_cols'])}，编码维度={X.shape[1]}。")

【INFO】【2025-12-12 21:45:06】【预处理】连续特征=8个，类别特征=35个
【INFO】【2025-12-12 21:45:07】【预处理】编码后维度=215
【INFO】【2025-12-12 21:45:07】【预处理】编码特征维度=215，样本数=101766
【INFO】【2025-12-12 21:45:07】【步骤3摘要】特征预处理完成：连续=8，类别=35，编码维度=215。


In [5]:
# 步骤4：构建桶树并检查划分
feature_cols_for_bucket = [c for c in df_raw.columns if c != target_col_model]

bucket_tree = BucketTree(
    cfg['BTTWD']['bucket_levels'],
    feature_names=feature_cols_for_bucket
)

bucket_ids_full = bucket_tree.assign_buckets(df_raw[feature_cols_for_bucket])

bucket_df = bucket_ids_full.value_counts().reset_index()
bucket_df.columns = ['bucket_id', 'count']

bucket_df['pos_rate'] = (
    df_raw.groupby(bucket_ids_full)[target_col_model]
    .apply(lambda s: (s == 1).mean())
    .values
)
display(bucket_df.head())
bucket_df.set_index('bucket_id')['count'].plot(kind='bar', figsize=(12,4), title='桶样本数分布')
fig_bucket = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'bucket_metrics_bar.png')
plt.savefig(fig_bucket, bbox_inches='tight')
plt.close()
log_info(f'【步骤4摘要】桶树划分完成，共有 {bucket_ids_full.nunique()} 个叶子桶。')


【INFO】【2025-12-12 21:45:08】【桶树】已为样本生成桶ID，共 112 个组合


,bucket_id,count,pos_rate
0,L1_number_inpatient=no_prev_inpt|L2_number_eme...,31852,0.000000
1,L1_number_inpatient=no_prev_inpt|L2_number_eme...,19768,0.000000
2,L1_number_inpatient=no_prev_inpt|L2_number_eme...,16861,0.242424
3,L1_number_inpatient=no_prev_inpt|L2_number_eme...,4697,0.166667
4,L1_number_inpatient=no_prev_inpt|L2_number_eme...,4477,0.245614


【INFO】【2025-12-12 21:45:11】【步骤4摘要】桶树划分完成，共有 112 个叶子桶。


In [6]:
# 步骤5：运行基线模型 k 折实验
# 基线部分在 run_kfold_experiments 内统一调度
log_info('【步骤5】基线模型将在整体交叉验证中一并运行。')
log_info('【步骤5摘要】基线模型性能将作为后续对比基准。')

【INFO】【2025-12-12 21:45:11】【步骤5】基线模型将在整体交叉验证中一并运行。
【INFO】【2025-12-12 21:45:11】【步骤5摘要】基线模型性能将作为后续对比基准。


In [7]:
import numpy as np
import pandas as pd

print("y 全局标签分布：", np.unique(y, return_counts=True))

print("原始标签列分布：")
print(df_raw[cfg['DATA']['target_col']].value_counts())


y 全局标签分布： (array([0, 1]), array([90409, 11357], dtype=int64))
原始标签列分布：
readmitted
0    90409
1    11357
Name: count, dtype: int64


In [9]:
# 步骤6：运行 BTTWD k 折实验（含基线）
results = run_kfold_experiments(X, y, df_raw.drop(columns=[cfg['DATA']['target_col']]), cfg)
summary_df = pd.read_csv(os.path.join(root_path, cfg['OUTPUT']['results_dir'], 'metrics_kfold_summary.csv'))
display(summary_df)
summary_df.plot(x='model', kind='bar', figsize=(8,4), title='模型指标对比')
fig_compare = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'metrics_compare.png')
plt.savefig(fig_compare, bbox_inches='tight')
plt.close()
log_info('【步骤6摘要】BTTWD 与基线的 k 折结果已生成并保存。')

【INFO】【2025-12-12 21:48:42】【基线-LogReg】使用决策阈值=0.300（fixed 模式）


d:\Anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
d:\Anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

【INFO】【2025-12-12 21:50:04】【基线-LogReg】整体指标：AUC_mean=0.669, AUC_std=0.007, BAC_mean=0.530, BAC_std=0.003, BND_ratio_mean=0.000, BND_ratio_std=0.000, F1_mean=0.127, F1_std=0.009, Kappa_mean=0.091, Kappa_std=0.010, MCC_mean=0.123, MCC_std=0.013, POS_Coverage_mean=nan, POS_Coverage_std=nan, Precision_mean=0.358, Precision_std=0.029, Recall_mean=0.077, Recall_std=0.006, Regret_mean=0.427, Regret_std=0.003
【INFO】【2025-12-12 21:50:04】【基线-RF】使用决策阈值=0.300（fixed 模式）


e:\yan\组\三支决策\机器学习\BT_TWD\bttwdlib\baselines.py:103: RuntimeWarning: Mean of empty slice
  summary[f"{col}_mean"] = float(np.nanmean(arr))
d:\Anaconda3\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


【INFO】【2025-12-12 21:51:30】【基线-RF】整体指标：AUC_mean=0.651, AUC_std=0.007, BAC_mean=0.536, BAC_std=0.002, BND_ratio_mean=0.000, BND_ratio_std=0.000, F1_mean=0.146, F1_std=0.005, Kappa_mean=0.106, Kappa_std=0.005, MCC_mean=0.137, MCC_std=0.006, POS_Coverage_mean=nan, POS_Coverage_std=nan, Precision_mean=0.367, Precision_std=0.013, Recall_mean=0.091, Recall_std=0.003, Regret_mean=0.423, Regret_std=0.002
【INFO】【2025-12-12 21:51:30】【基线-KNN】使用决策阈值=0.300（fixed 模式）


...

d:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:697: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
d:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:697: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
d:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:697: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
d:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:697: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
d:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:697: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
d:\Anaconda3\Lib\site-packages\sklearn\metrics\_classif

【INFO】【2025-12-12 21:53:24】[BASELINE] 桶 L1_number_inpatient=no_prev_inpt|L2_number_emergency=no_prev_er|L3_discharge_disposition_id=Home|L4_number_diagnoses=high_dx: BAC=0.5000, Regret=0.3265, Precision=0.0000, Recall=0.0000
【INFO】【2025-12-12 21:53:24】[BASELINE] 桶 L1_number_inpatient=no_prev_inpt|L2_number_emergency=no_prev_er|L3_discharge_disposition_id=OtherDisp|L4_number_diagnoses=mid_dx: BAC=0.5930, Regret=0.3108, Precision=0.6154, Recall=0.2000
【INFO】【2025-12-12 21:53:24】[BASELINE] 桶 L1_number_inpatient=no_prev_inpt|L2_number_emergency=no_prev_er|L3_discharge_disposition_id=OtherDisp|L4_number_diagnoses=high_dx: BAC=0.5373, Regret=0.4522, Precision=0.5652, Recall=0.0878
【INFO】【2025-12-12 21:53:24】[BASELINE] 桶 L1_number_inpatient=no_prev_inpt|L2_number_emergency=no_prev_er|L3_discharge_disposition_id=Transfer|L4_number_diagnoses=high_dx: BAC=0.5024, Regret=0.5212, Precision=0.5000, Recall=0.0056
【INFO】【2025-12-12 21:53:24】[BASELINE] 桶 L1_number_inpatient=no_prev_inpt|L2_number_emer

d:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:697: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)


,model,Precision_mean,Precision_std,Recall_mean,Recall_std,F1_mean,F1_std,BAC_mean,BAC_std,AUC_mean,...,MCC_mean,MCC_std,Kappa_mean,Kappa_std,BND_ratio_mean,BND_ratio_std,POS_Coverage_mean,POS_Coverage_std,Regret_mean,Regret_std
0,BTTWD,0.275114,0.011300,0.242319,0.012921,0.257519,0.010254,0.581014,0.005510,0.676102,...,0.171369,0.010872,0.170875,0.010911,0.148419,0.00543,0.01815,0.007188,0.393353,0.004314
1,LogReg,0.357968,0.028869,0.077397,0.006008,0.127210,0.009444,0.529955,0.003162,0.669471,...,0.122930,0.013118,0.091111,0.009546,0.000000,0.00000,NaN,NaN,0.427382,0.003136
2,RandomForest,0.366786,0.013253,0.091221,0.003174,0.146084,0.004819,0.535711,0.001665,0.651329,...,0.136902,0.006437,0.106347,0.004922,0.000000,0.00000,NaN,NaN,0.423265,0.001668
3,KNN,0.174797,0.007325,0.221185,0.010735,0.195245,0.008468,0.545007,0.005332,0.567914,...,0.081385,0.009514,0.080645,0.009416,0.000000,0.00000,NaN,NaN,0.464192,0.005247
4,XGBoost,0.387136,0.022915,0.098354,0.003774,0.156824,0.006376,0.539372,0.002322,0.678575,...,0.149406,0.010335,0.116842,0.007240,0.000000,0.00000,NaN,NaN,0.419914,0.002598


【INFO】【2025-12-12 21:53:25】【步骤6摘要】BTTWD 与基线的 k 折结果已生成并保存。


In [10]:
# 步骤7：桶级别分析
bucket_metrics_path = os.path.join(root_path, cfg['OUTPUT']['results_dir'], 'bucket_metrics.csv')
if os.path.exists(bucket_metrics_path):
    bucket_metrics_df = pd.read_csv(bucket_metrics_path)
    display(bucket_metrics_df.head())
    bucket_metrics_df.plot(x='bucket_id', y='pos_rate_all', kind='bar', figsize=(12,4), title='桶正类比例')
    plt.ylabel('正类比例')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig(fig_bucket, bbox_inches='tight')
    plt.close()
log_info('【步骤7摘要】桶级指标已整理，可用于局部化分析。')

,bucket_id,layer,parent_bucket_id,n_train,n_val,pos_rate_train,pos_rate_val,alpha,beta,regret_val,...,is_weak,threshold_source_bucket,parent_with_threshold,n_test,pos_rate_test,BND_ratio_test,POS_Coverage_test,regret_test,fold,pos_rate
0,ROOT,L1,NaN,11397,4884,0.110994,0.111179,0.4,0.2,0.393837,...,False,ROOT,NaN,NaN,NaN,NaN,NaN,NaN,1,0.111593
1,L1_number_inpatient=no_prev_inpt,L1,ROOT,9770,4318,0.092323,0.093330,0.4,0.2,0.360931,...,False,L1_number_inpatient=no_prev_inpt,NaN,NaN,NaN,NaN,NaN,NaN,1,0.094130
2,L1_number_inpatient=no_prev_inpt|L2_number_eme...,L2,L1_number_inpatient=no_prev_inpt,9560,4204,0.091109,0.096337,0.4,0.2,0.369767,...,True,L1_number_inpatient=no_prev_inpt,L1_number_inpatient=no_prev_inpt,NaN,NaN,NaN,NaN,NaN,1,0.093029
3,L1_number_inpatient=no_prev_inpt|L2_number_eme...,L3,L1_number_inpatient=no_prev_inpt|L2_number_eme...,29008,12336,0.073394,0.077983,0.5,0.4,0.311933,...,False,L1_number_inpatient=no_prev_inpt|L2_number_eme...,NaN,NaN,NaN,NaN,NaN,NaN,1,0.074763
4,L1_number_inpatient=no_prev_inpt|L2_number_eme...,L3,L1_number_inpatient=no_prev_inpt|L2_number_eme...,13892,6040,0.120933,0.124503,0.4,0.2,0.480712,...,True,L1_number_inpatient=no_prev_inpt,L1_number_inpatient=no_prev_inpt,NaN,NaN,NaN,NaN,NaN,1,0.122015


C:\Users\xzq20.HUANLI\AppData\Local\Temp\ipykernel_13592\946468196.py:9: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()


【INFO】【2025-12-12 21:53:27】【步骤7摘要】桶级指标已整理，可用于局部化分析。


In [ ]:
# 步骤8：结果汇总
log_info('【步骤8】检查结果文件与图表。')
print(os.listdir(os.path.join(root_path, cfg['OUTPUT']['results_dir'])))
print(os.listdir(os.path.join(root_path, cfg['OUTPUT']['figs_dir'])))
log_info('【全部步骤完成】Telco Churn 数据集上的 BT-TWD 实验结束。')

【INFO】【2025-12-12 21:53:27】【步骤8】检查结果文件与图表。
['baseline_bucket_metrics.csv', 'bucket_fallback_stats.csv', 'bucket_metrics.csv', 'bucket_metrics_gain.csv', 'bucket_thresholds.csv', 'bucket_thresholds_per_fold.csv', 'bucket_tree_structure.csv', 'metrics_kfold_per_fold.csv', 'metrics_kfold_summary.csv', 'metrics_overview.csv']
['bank_class_distribution.png', 'bucket_metrics_bar.png', 'class_distribution.png', 'metrics_compare.png']
【INFO】【2025-12-12 21:53:27】【全部步骤完成】Telco Churn 数据集上的 BT-TWD 实验结束。


: 